In [50]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from newspaper import Article
import json
from string import digits
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.stem.porter import *
from collections import Counter 
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech

In [2]:
# data load from muckrack
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)

In [64]:
del writer_df

In [3]:
# Establish connection with database
client = MongoClient()
mydb = client["testinsightdb"]

In [4]:
currcol = mydb['zachlowe_nba']
y = currcol.find({"name": 'zachlowe_nba'})

In [60]:
tmpR = []
for i in range(0,100):
    tmpR.append(sys.getsizeof(y[i]['article']))

In [62]:
np.mean(tmpR)

20471.17

In [51]:
# preprocessing functions
def clean_text(txtstr):
    txtstr = re.sub(r'\n\s*\n', '', txtstr) # extra lines
    txtstr = re.sub(r'[^A-Za-z0-9(),!?@\'\`\"\_\n]', ' ', txtstr)
    txtstr = re.sub(r'[^\w\s]','',txtstr) # punctuation
    txtstr = re.sub(r'\w*\d\w*','',txtstr) # words with number plust letters
    remove_digits = str.maketrans('', '', digits)
    txtstr = txtstr.translate(remove_digits)
    txtstr = re.sub(' +',' ',txtstr) # extra white spaces
    txtstr = txtstr.lower()
    return txtstr

# stem long string - must be tokenized
def stemstr(txtstr):
    ps = PorterStemmer()
    newstr = [ps.stem(curr_word) for curr_word in txtstr]
    return newstr

# for figuring out what type of work
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0] # first indexer for 
            # getting the top POS from list, second indexer for getting POS from tuple( POS: count )

# lemmatize long string - must be tokenized
def lemstr(txtstr):
    lz = WordNetLemmatizer()
    newstr = [lz.lemmatize(curr_word, get_pos(curr_word)) for curr_word in txtstr]
    return newstr

In [42]:
# article_df = y[2]
# article_df['article']

In [14]:
a = clean_text(article_df['article'])
a

'jimmy butler in june fell right into the nebulous region where debating a maximum or new supermaximum contract becomes especially fraught for teams with another all nba berth the next season butler would qualify for a five year million supermax vaulting to a salary equivalent to percent of the salary cap percentage points more than normal for a player at his experience level the difference would amount to million or more over the full deal add annual percent raises that might outpace growth in the cap and such a deal could soak up percent of a teams cap space on the back end it is as large a commitment as exists in the nba butler was the only star on a mediocre team without finding a second and perhaps a third the bulls would go nowhere interesting during the length of butlers supermax the sheer size of his deal could make luring those stars harder butler was about to turn depending on your taste he at that point was somewhere between the eighth and best player in the league you do th

In [22]:
b = sent_tokenize(article_df['article'])
c = word_tokenize(a)
c

['jimmy',
 'butler',
 'in',
 'june',
 'fell',
 'right',
 'into',
 'the',
 'nebulous',
 'region',
 'where',
 'debating',
 'a',
 'maximum',
 'or',
 'new',
 'supermaximum',
 'contract',
 'becomes',
 'especially',
 'fraught',
 'for',
 'teams',
 'with',
 'another',
 'all',
 'nba',
 'berth',
 'the',
 'next',
 'season',
 'butler',
 'would',
 'qualify',
 'for',
 'a',
 'five',
 'year',
 'million',
 'supermax',
 'vaulting',
 'to',
 'a',
 'salary',
 'equivalent',
 'to',
 'percent',
 'of',
 'the',
 'salary',
 'cap',
 'percentage',
 'points',
 'more',
 'than',
 'normal',
 'for',
 'a',
 'player',
 'at',
 'his',
 'experience',
 'level',
 'the',
 'difference',
 'would',
 'amount',
 'to',
 'million',
 'or',
 'more',
 'over',
 'the',
 'full',
 'deal',
 'add',
 'annual',
 'percent',
 'raises',
 'that',
 'might',
 'outpace',
 'growth',
 'in',
 'the',
 'cap',
 'and',
 'such',
 'a',
 'deal',
 'could',
 'soak',
 'up',
 'percent',
 'of',
 'a',
 'teams',
 'cap',
 'space',
 'on',
 'the',
 'back',
 'end',
 'it',

In [55]:
d = word_tokenize(b[0])
print(d)
print(lemstr(['fell', 'fall', 'flew', 'flies']))

['Jimmy', 'Butler', 'in', 'June', '2017', 'fell', 'right', 'into', 'the', 'nebulous', 'region', 'where', 'debating', 'a', 'maximum', 'or', 'new', 'supermaximum', 'contract', 'becomes', 'especially', 'fraught', 'for', 'teams', '.']
['fell', 'fall', 'fly', 'fly']


In [56]:
clean_text(b[0])

'jimmy butler in june fell right into the nebulous region where debating a maximum or new supermaximum contract becomes especially fraught for teams '